In [151]:
import numpy as np
import pandas as pd
import itertools
import operator
from haversine import haversine
import time

#set cd to data folder with QC'ed rides
% cd /Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data/


/Users/fineiskid/Desktop/Access_Analysis_Rproject_local/data


In [2]:
data = pd.read_csv("single_clean_day.csv")

In [3]:
data["ServiceDate"] = pd.to_datetime(data["ServiceDate"], format ='%Y-%m-%d %H:%M:%S', unit = "D")

In [4]:
def add_TimeWindows(data, windowsz):
    '''calculate time windows (pickup and dropoff)
        from SchTime and ETA.
        data is subsetted schedule data from a day.
        windowsz is size of pickup/dropoff window in seconds'''

    etas = data["ETA"]
    schtime =data["SchTime"]
    schtime[schtime<0] = np.nan
    data["PickupStart"] = 0; data["PickupEnd"] = 0
    data["DropoffStart"] = 0; data["DropoffEnd"] = 0
    for x in range(0, len(etas)):

        #make dropoff window when there's no required drop off time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] <0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ETA"].iloc[x]+3600
        
        #make dropoff window when there IS a required drop off time: 1hr before ReqLate time
        if (data["Activity"].iloc[x] == 1) & (data["ReqLate"].iloc[x] >0):
            data["DropoffStart"].iloc[x] = data["ETA"].iloc[x]-3600
            data["DropoffEnd"].iloc[x] = data["ReqLate"].iloc[x]  
        
        #schtime is in the middle of the pick up window
        if data["Activity"].iloc[x] == 0:
            data["PickupStart"].iloc[x] = schtime.iloc[x]-(windowsz/2)
            data["PickupEnd"].iloc[x] = schtime.iloc[x]+(windowsz/2)

    return data

In [20]:
# SAVE SINGLE DAY'S RIDES WITH TIME WINDOWS
data = add_TimeWindows(data, 60*30)

from contextlib import contextmanager
import sys, os

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


with suppress_stdout():
    data.to_csv(path = "/Users/fineiskid/Desktop/single_day_TimeWindows.csv", index = False, quoting=None)

In [21]:
def get_URIDs(data, broken_Run, resched_init_time):
    '''get unscheduled request id's from broken bus,
        based on when we're allowed to first start rescheduling.
        resched_init_time is in seconds, marks the point in time we can begin considering reinserting new requests.
        broken_Run is number of run that breaks
        data is today's scheduling data

        RETURN: list of URIDs'''
    
    #all rides that exist past time we're allowed to begin rescheduling
    leftover = data[data["ETA"] >= resched_init_time]
    leftover = leftover[(leftover["Activity"] != 6) & (leftover["Activity"] != 16) & (leftover["Activity"] != 3)]
    
    #rides that were scheduled to be on broken bus past resched_init_time
    pickmeup = leftover[leftover["Run"]==broken_Run]
    clients = pickmeup["ClientId"].unique()
    clients = clients[~(np.isnan(clients))]
    rmClients = []
    
    #remove people who would were scheduled to be on bus before resched_init_time
    for cli in clients:
        onoff = pickmeup[pickmeup["ClientId"]==cli]
        if onoff.shape[0] == 1:
            rmClients.append(cli)
    unsched = pickmeup[~pickmeup["ClientId"].isin(rmClients)]

    print("There are %s rides left to be scheduled on broken run %s" % (unsched.shape[0], broken_Run))

    class URID:
        def __init__(self, BookingId, Run, PickUpCoords, DropOffCoords, PickupStart, PickupEnd, DropoffStart, DropoffEnd, SpaceOn, MobAids):
            self.BookingId= BookingId
            self.Run = Run
            self.PickUpCoords = PickUpCoords
            self.DropOffCoords = DropOffCoords
            self.PickupStart = PickupStart
            self.PickupEnd = PickupEnd
            self.DropoffStart = DropoffStart
            self.DropoffEnd = DropoffEnd
            self.SpaceOn = SpaceOn
            self.MobAids = MobAids


    diffIDs = unsched.BookingId.unique()
    saveme = []

    #save separate URID's in a list
    for ID in diffIDs:
        temp = URID(BookingId = ID,
            Run = broken_Run,
            PickUpCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[0,],
            DropOffCoords = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["LAT", "LON"]].iloc[1,],
            PickupStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupStart"]].iloc[0,],
            PickupEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["PickupEnd"]].iloc[0,],
            DropoffStart = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffStart"]].iloc[0,],
            DropoffEnd = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["DropoffEnd"]].iloc[0,],
            SpaceOn = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["SpaceOn"]].iloc[0,],
            MobAids = unsched[unsched["BookingId"]==unsched.BookingId.iloc[0]][["MobAids"]].iloc[0,])
        saveme.append(temp)

    return saveme

In [22]:
broken_Run = data.Run.unique()[0]
resched_init_time = 14*60*60
urids = get_URIDs(data, broken_Run, resched_init_time)

There are 10 rides left to be scheduled on broken run 680SEB


In [23]:
def get_busRuns(data, Run, resched_init_time):
    ''' take pd.DataFrame from add_Time_Windows.py and create busRun object for specified Run number,
    for all stops occurring at or after resched_init_time.

    RETURN: busRun object for specified Run.'''

    #subset based on matching Run number, and subset for stops only at or after resched_init_time
    print("Getting remaing rides for run %s" % Run)
    dataSub = data[(data["Run"] == Run)] # & (data["ETA"] >= resched_init_time)]
    #subset only the rides that aren't 6, 16, or 3:
    leave = dataSub.index.min()
    gas = dataSub[(dataSub["Activity"]==6)|(dataSub["Activity"]==16)|(dataSub["Activity"]==3)].index.min()
    busRun = dataSub.loc[leave:(gas-1)]

    return busRun

In [24]:
#Test get_busRuns subsetting routine
testRun = data.Run.unique()[2]
bus181SEB = get_busRuns(data, testRun, resched_init_time)


Getting remaing rides for run 181SEB


In [75]:
#list urid attributes:
urids[0].__dict__.keys()

['PickupEnd',
 'Run',
 'BookingId',
 'MobAids',
 'DropoffStart',
 'DropoffEnd',
 'PickUpCoords',
 'SpaceOn',
 'DropOffCoords',
 'PickupStart']

In [105]:
#Find windows that overlap 
def time_overlap(Run_Schedule, URID, pickUpDropOff = True):
    '''URID: of class URID, has bookingId, pickUpLocation, dropOffLocation, etc.
        Run_Schedule: Schedule (pd.Data.Frame) of the run on which we're trying to insert the URID
        RETURN: dictionary containing indices of schedule-outbound and -inbound nodes that we need
        to get distance between w/r/t URID location.'''

    #How it works: first, find all nodes that have time overlap with the URID's (pickup or dropoff) window
    # second, notice any gaps in the order of these nodes from the original bus ride.
    # For the first chunk of overlapping nodes, add the lower-time bound node in, given that the first node
    # even exists in the Run_Schedule.
    # For every chunk of contiguous overlap nodes, add in the upper-time bound node, because there can potentially
    # be an inbound ride from the URID node to the upper-time bound node. There can't be an outbound ride.

    if pickUpDropOff:
        Start = URID.PickupStart[0]
        End = URID.PickupEnd[0]
    else:
        Start = URID.DropoffStart[0]
        End = URID.DropoffEnd[0]
        
    crossover = []
    
    for jj in range(Run_Schedule.shape[0]):
        #Checking if a Run's PickupWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 0:
            PUE = Run_Schedule.PickupEnd.iloc[jj]; PUS = Run_Schedule.PickupStart.iloc[jj]
            #simple, unequal overlap
            if (PUE > Start) & (PUS < End):
                crossover.append(Run_Schedule.index[jj])
            # equal or strictly within [PUS, PUE]
            if (PUE <= End) & (PUS >= Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered by [PUS, PUE] and then some on both sides
            if (PUE > End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on left side
            if (PUE == End) & (PUS < Start):
                crossover.append(Run_Schedule.index[jj])
            # [Start, End] completely covered and then some only on right side
            if (PUS == Start) & (PUE > End):
                crossover.append(Run_Schedule.index[jj])
                
        #Checking if a Run's DropoffWindow overlaps with URID's Window.
        if Run_Schedule.Activity.iloc[jj] == 1:
            DOE = Run_Schedule.DropoffEnd.iloc[jj]; DOS = Run_Schedule.DropoffStart.iloc[jj]
            if (DOE > Start) & (DOS < End):
                crossover.append(Run_Schedule.index[jj])
            if (DOE <= End) & (DOS >= Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE > End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOE == End) & (DOS < Start):
                crossover.append(Run_Schedule.index[jj])
            if (DOS == Start) & (DOE > End):
                crossover.append(Run_Schedule.index[jj])
                
    #Get rid of cases that repeat themselves:
    crossover = list(set(crossover))
    
    inserts = Run_Schedule.loc[crossover]; indices = Run_Schedule.index
    lst = []; outbound = []; inbound = []
    #Lists of continuously arranged nodes with overlap
    for k, g in itertools.groupby(enumerate(inserts.index), lambda (i,x):i-x):
        k = map(operator.itemgetter(1), g)
        #save nodes with time overlap
        outbound += k; inbound +=k
        #save upper/lower bound where appropriate
        lst += [min(k)]; lst+=[max(k)]
        if len(lst) == 2:
            if lst[0] != min(indices):
                outbound.append(lst[0]-1) #we have a lower bound node, heading outbound from Run_Schedule
            if lst[-1] != max(indices):
                inbound.append(lst[-1]+1) #upper bound node for first contiguous set of overlap nodes
        else:
            if lst[-1] != max(indices):
                inbound.append(lst[-1]+1) #upper bound node for first contiguous set of overlap nodes
    
    outbound, inbound = map(sorted, [outbound, inbound])
    #print("Need to service URID within %s sec to %s sec" % (Start, End))
    #print("These indices of Run_Schedule will need\nto have distances calculated: %s" % np.union1d(outbound, inbound))

    retDict = {"outbound": outbound, "inbound": inbound, "all_nodes" : sorted(np.union1d(outbound, inbound))}
    return retDict

In [144]:
#print distances that bus 181SEB is from URID location during time windows... It's pretty far...
def radius_Elimination(data, URID, radius, pickUpDropOff):
    '''Given a set of the day's bus data and an unhandled requst,
    eliminate all bus routes that are farther than radius-miles away at all
        points in the URID's pickup or dropoff window, the window being specified
        by pickUpDropOff. Run time_insertions.py on the resultant list.

        NOTE: you need to "> pip install haversine"

        INPUT:  data - pd.Data.Frame returned from add_TimeWindows.py
                URID - URID object from get_URIDS.py	
                radius - float, number of miles
                pickUpDropOff - boolean True/False for PickUp (True) or Dropoff (False)

        RETURN: LIST of runs within radius-miles of URID.'''

    #obviously, broken bus can't be in the list of nearby buses.
    data = data[data.Run != URID.Run]

    if pickUpDropOff:
        URID_loc = ([URID.PickUpCoords["LAT"], URID.PickUpCoords["LON"]])
    else:
        URID_loc = ([URID.DropOffCoords["LAT"], URID.PickUpCoords["LON"]])
        
    #get pd.Data.Frame of nodes that have overlap with URID's pickup or dropoff window
    overlap_data = time_overlap(data, URID, pickUpDropOff)

    #get row index of nodes that may have either inbound/outbound overlap with URID TW.
    overlap_data = data.loc[overlap_data['all_nodes']]
    overlap_LAT = overlap_data.LAT.tolist()
    overlap_LON = overlap_data.LON.tolist()

    #store list of rides that are sufficiently nearby URID's location
    okBuses = []
    for k in range(len(overlap_LAT)):
        point = (overlap_LAT[k], overlap_LON[k])
        dist = haversine(point, URID_loc, miles=True)
        if(dist < radius):
            okBuses.append(overlap_data.Run.iloc[k])

    return set(okBuses)

In [163]:
radius = 5 #miles cutoff

#list of buses to plug URID onto:
good_buses = radius_Elimination(data, URID, 5, True)
print("At 1:30 PM, there are %s buses\nwithin %s miles that have nodes that\noverlap with URID." %
      (len(good_buses), radius))

At 1:30 PM, there are 64 buses
within 5 miles that have nodes that
overlap with URID.


In [152]:
#Get distances between (insertable nodes, URID location)
import requests
def osrm (URID_location, inbound, outbound):
    #URID_location it's a list: [lat, lon]
    #lists for inbound and outbound matrices
    # inbound/outbound: 2-column np.arrays storing inbound/outbound node latitude/longitude
    # and inbound (from scheduled location to urid location) 
    out_total_time = []
    out_start_points = []
    out_end_points = []
    in_total_time = []
    in_start_points = []
    in_end_points = []
    out_osrm_url = "http://router.project-osrm.org/viaroute?"
    in_osrm_url = "http://router.project-osrm.org/viaroute?"
    urid_LAT = URID_location[0]; urid_LON = URID_location[1]

    # outbound
    for lat_cord,lon_cord in outbound: 
        out_route_url = out_osrm_url+ "loc=" + str(lat_cord) + "," + str(lon_cord)
        out_route_url = out_route_url + "&loc=" + str(urid_LAT) + "," + str(urid_LON) + "&instructions=false"
        out_route_requests = requests.get(out_route_url)
        out_route_results = out_route_requests.json()
        out_total_time += [out_route_results[u'route_summary'][u'total_time']]
        out_start_points += [out_route_results[u'route_summary'][u'start_point']]
        out_end_points += [out_route_results[u'route_summary'][u'end_point']]

    # inbound
    for lat_cord,lon_cord in inbound: 
        in_route_url = in_osrm_url + "loc=" + str(urid_LAT) + "," + str(urid_LON) 
        in_route_url = in_route_url+ "&loc=" + str(lat_cord) + "," + str(lon_cord) + "&instructions=false"
        in_route_requests = requests.get(in_route_url)
        in_route_results = in_route_requests.json()
        in_total_time += [in_route_results[u'route_summary'][u'total_time']]
        in_start_points += [in_route_results[u'route_summary'][u'start_point']]
        in_end_points += [in_route_results[u'route_summary'][u'end_point']]

    a = np.array([in_total_time]); b = np.array([out_total_time])
    retDict = {"inbound_times": a, "outbound_times": b}
    
    return(retDict)


In [160]:
#Time the travel time matrix function:

#formulate URID's location:
uridLoc = [URID.PickUpCoords[0], URID.PickUpCoords[1]]

#Get nodes from 181SEB bus that overlap with URID
pickUpDropOff = True; URID = urids[0]; Run_Schedule = bus181SEB

#formulate inbound nodes and outbound nodes, for input into OSRM!!!
inserts = time_overlap(Run_Schedule, URID, pickUpDropOff)
outbound = Run_Schedule.loc[inserts["outbound"]]
outbound = np.column_stack((np.array(outbound.LAT), np.array(outbound.LON))) #GODDAMNIT THAT WAS COMPLICATED
inbound = Run_Schedule.loc[inserts["inbound"]]
inbound = np.column_stack((np.array(inbound.LAT), np.array(inbound.LON)))

#print("inbound nodes: %s" % inbound)
#print("outbound nodes: %s" % outbound)

start = time.clock()
time_matrix = osrm(uridLoc, inbound, outbound)
elapsed = time.clock()-start
print("API call took %s sec" % elapsed)

API call took 0.081226 sec


In [162]:
#We now have travel times for the inbound, outbound insertions!
outbound_times = np.column_stack((outbound, time_matrix["outbound_times"].T))
inbound_times = np.column_stack((inbound, time_matrix["inbound_times"].T))
outbound_times

array([[   47.71,  -122.19,  2058.  ],
       [   47.7 ,  -122.2 ,  2162.  ],
       [   47.69,  -122.18,  2045.  ],
       [   47.76,  -122.15,  2515.  ],
       [   47.75,  -122.2 ,  2415.  ],
       [   47.75,  -122.2 ,  2415.  ],
       [   47.72,  -122.2 ,  2182.  ],
       [   47.68,  -122.16,  2050.  ],
       [   47.7 ,  -122.16,  2200.  ]])

###Cool, now we have everything we need to start checking feasibility:
1. We have ``good_buses``: list of buses within radius that have stops overlapping with the URID's time window. Don't worry, the broken bus isn't on this list.

2. We have ``Run_Schedule``: a Pandas dataframe that has all stops relevant to squeezing in URID.

3. We have ``inserts``: dictionary with insertion nodes on the scheduled buses that have overlapping time windows.

4. We have ``outbound_times`` and ``inbound_times``: matrices travel times between the chosen scheduled bus's insertion nodes and the URID's location.
